to do:
train_X :
- change label to 0/1 (Normal/Attack)
- Normalize data
- Split to matrix (60 * var_length) = 60 because 1 hour
- transpose matrix to (var_lenth * 60)

train_Y :
- split to matrix @60
- if in the matrix there is 1 (attack) make label as 1 

Input shape:
(49990, 53)
Expected output shape:
train_X: (7498, 52, 60)
train_Y: (7498, 1)

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [3]:
data = pd.read_csv('SWaT_Dataset_Attack_v0_cleaned.csv')
# df = data['Timestamp','FIT101','LIT101','MV101','P101','P102','AIT201','AIT202','AIT203','FIT201','MV201','P201','P202','P203','P204','P205','P206','DPIT301','FIT301','LIT301','MV301','MV302','MV303','MV304','P301','P302','AIT401','AIT402','FIT401','LIT401','P401','P402','P403','P404','UV401','AIT501','AIT502','AIT503','AIT504','FIT501','FIT502','FIT503','FIT504','P501','P502','PIT501','PIT502','PIT503','FIT601','P601','P602','P603','Normal/Attack']
data.info

<bound method DataFrame.info of                       Timestamp    FIT101    LIT101   MV101  P101  P102  \
0        28/12/2015 10:00:00 AM  2.427057  522.8467       2     2     1   
1        28/12/2015 10:00:01 AM  2.446274  522.8860       2     2     1   
2        28/12/2015 10:00:02 AM  2.489191  522.8467       2     2     1   
3        28/12/2015 10:00:03 AM  2.534350  522.9645       2     2     1   
4        28/12/2015 10:00:04 AM  2.569260  523.4748       2     2     1   
...                         ...       ...       ...     ...   ...   ...   
449914      2/1/2016 2:59:55 PM  2.559972  519.5495       2     2     1   
449915      2/1/2016 2:59:56 PM  2.549082  520.4131       2     2     1   
449916      2/1/2016 2:59:57 PM  2.531467  520.6878       2     2     1   
449917      2/1/2016 2:59:58 PM  2.521218  520.7271       2     2     1   
449918      2/1/2016 2:59:59 PM  2.501681  521.1196       2     2     1   

          AIT201    AIT202    AIT203    FIT201  ...  P501  P502    

In [4]:
data.head()

,Timestamp,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,...,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603,Normal/Attack
0,28/12/2015 10:00:00 AM,2.427057,522.8467,2,2,1,262.0161,8.396437,328.6337,2.445391,...,2,1,250.8652,1.649953,189.5988,0.000128,1,1,1,Normal
1,28/12/2015 10:00:01 AM,2.446274,522.8860,2,2,1,262.0161,8.396437,328.6337,2.445391,...,2,1,250.8652,1.649953,189.6789,0.000128,1,1,1,Normal
2,28/12/2015 10:00:02 AM,2.489191,522.8467,2,2,1,262.0161,8.394514,328.6337,2.442316,...,2,1,250.8812,1.649953,189.6789,0.000128,1,1,1,Normal
3,28/12/2015 10:00:03 AM,2.534350,522.9645,2,2,1,262.0161,8.394514,328.6337,2.442316,...,2,1,250.8812,1.649953,189.6148,0.000128,1,1,1,Normal
4,28/12/2015 10:00:04 AM,2.569260,523.4748,2,2,1,262.0161,8.394514,328.6337,2.443085,...,2,1,250.8812,1.649953,189.5027,0.000128,1,1,1,Normal


- change label to 0/1 (Normal/Attack)

In [5]:
data['label'] = pd.factorize(data['Normal/Attack'])[0].astype(np.uint16)
data['label'].unique()

array([0, 1], dtype=uint16)

- change timestamp to datetime format

In [6]:
# separate the date
train_dates = pd.to_datetime(data[' Timestamp'])

In [7]:
# separate the label
train_label = data['label']
#drop timestamp, label, and ex label column
data = data.drop(['Normal/Attack', ' Timestamp', 'label'], axis=1)
# change type to float
data = data.astype(float, errors = 'raise')
data.shape

(449919, 51)

In [8]:
# normalize the data
scaler = StandardScaler()
scaler = scaler.fit(data)
df_for_training_scaled = scaler.transform(data)

In [9]:
# delete last 39 rows, because we want to reshape to 60 length 
df_for_training_scaled = df_for_training_scaled[:-39, :]
df_for_training_scaled.shape

(449880, 51)

In [10]:
# Transpose data before splitting
transposed_data = df_for_training_scaled.T
transposed_data.shape

(51, 449880)

In [11]:
splitted_data = np.hsplit(transposed_data,7498) #equally 60 length
X_train = np.asarray(splitted_data)
X_train.shape

(7498, 51, 60)

now create the label train_Y

In [12]:
train_label = np.asarray(train_label)
train_label = train_label[:-39] #remove last 39 labels
splitted_train_label = np.split(train_label,7498) #split to equally 60
yarr = np.asarray(splitted_train_label)
yarr.shape

(7498, 60)

In [13]:
def linear_search(values, search_for):
   search_at = 0
   search_res = False
# Match the value with each data element	
   while search_at < len(values) and search_res is False:
      if values[search_at] == search_for:
         search_res = True
      else:
         search_at = search_at + 1
   return search_res

In [14]:
Y_train = []
for i in range(len(yarr)):
    attacked = linear_search(yarr[i], 1) #if we find 1 in 60 label (1 hour), true = attacked, false = normal
    Y_train.append(attacked)

Y_train = np.array(Y_train).astype('uint8')

In [15]:
Y_train = np.reshape(Y_train, (len(Y_train), 1)) #reshape to match the input for cnn
Y_train.shape

(7498, 1)

In [16]:
train_X, test_X, train_Y, test_Y = train_test_split(X_train, Y_train, test_size=0.25, random_state=42, shuffle=False)

In [17]:
print("train_X shape = ", train_X.shape)
print("train_Y shape = ", train_Y.shape)
print("test_X shape = ", test_X.shape)
print("test_Y shape = ", test_Y.shape)

train_X shape =  (5623, 51, 60)
train_Y shape =  (5623, 1)
test_X shape =  (1875, 51, 60)
test_Y shape =  (1875, 1)


In [18]:
# save numpy array as npy file
from numpy import save
# save to npy file
save('X_train.npy', train_X)
save('y_train.npy', train_Y)
save('X_test.npy', test_X)
save('y_test.npy', test_Y)